In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot

In [2]:
data = pd.read_excel("./data/1_train.xlsx")
data.head(5)

,主页ID,主页名称,分类,message,name,description,发文类型,y
0,155000000000000,纬来体育台官方粉丝团,电视频道,中华队长林智胜 三分砲！！！\n人如其名 真的「致胜」阿～,NaN,NaN,video,H
1,359000000000,公视粉丝团,电视频道,您所选择的电视频道，将会决定您的未来~\n遥控器的决定权，就在你手上！\n\n#看见更好的未来,2015公视电视募款 公视让你看见更好的未来,这是一个资讯爆炸的时代，你所选择的资讯将决定你的视野；你所选择的电视频道，将会决定你的未来 ...,video,L
2,359000000000,公视粉丝团,电视频道,好演员值得更多肯定和掌声！\n\n吴慷仁提到大概有六、七年资历的演员，所面临的最大困境，是赚...,公视粉丝团,NaN,link,L
3,190000000000,Mobile01,网路社群,大师兄 一发炸裂！！帅啊啊啊啊啊啊啊！！！,来自 Mobile01 贴文的相片,NaN,photo,H
4,359000000000,公视粉丝团,电视频道,「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不到鹿港的味道了！」\n\n「两...,【我们的岛】抢救鹿港,叶明兰、叶镇中 / 采访报导 「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不...,link,L


In [3]:
from Python.pandas.feature_selection import *
object_feature_helper(data)


---------------------------------------------
data contains [ 7829 ] records 
主页ID do not have any null values.
主页ID  is numeric column:
count    7.829000e+03
mean     2.764040e+14
std      4.093619e+14
min      6.282507e+10
25%      1.090000e+14
50%      1.260000e+14
75%      2.420000e+14
max      1.650000e+15
Name: 主页ID, dtype: float64



---------------------------------------------
主页名称 do not have any null values.
主页名称  has [ 133 ] different values, please create features yourself!
please notice here are some seldom_indexes:  Index(['疯神无双', 'PTT网路电台/PTTradio', '我的自由年代', '金马影展 TGHFF', '联合报plus', '十二夜',
       '沃草！（Watchout）', 'NOWnews.com 今日新闻网', '【玩命关头】电影粉丝专属页面', '苹果娱乐新闻',
       'Running Man 台湾之家', '长不大的爸爸',
       '老外看中国、老外看台湾 A Laowai's View of China & Taiwan', 'PanSci 科学新闻网',
       'udn.com 联合新闻网', '大纪元时报 - 台湾(The Epoch Times - Taiwan)',
       '得利影视．go movie 情报站', '新头壳newtalk', 'KKBOX', 'ifilm/传影互动', '听见幸福',
       '食尚玩家', 'PNN公视新闻议题中心', '综艺大集合', '妹妹', 'MamaClub',
       'PT

主页ID 提取 blacklist, whitelist, data_hl_score

blacklist 和 whitelist 不加入训练名单, 直接判定。
419000000000000 只有一个正例，也加入黑名单。
data_hl_score 只对有50条记录以上Id计算，少于50条的记为0。
data_hl_score = high_count / total_count

In [4]:
grp = data.pivot_table(index='主页ID', columns='y', values=['分类'], aggfunc='count').droplevel(0, axis=1)
grp = grp.fillna(0)
blacklist = grp[((grp['H'] <= 1) & (grp['L'] > 5))]
whitelist = grp[(grp['L'] == 0) & (grp['H'] > 1)]

In [5]:
data_remove_bw = data[(~data['主页ID'].isin(blacklist.index.values)) & (~data['主页ID'].isin(whitelist.index.values))]
data_remove_bw = data_remove_bw.reset_index()
data_remove_bw

,index,主页ID,主页名称,分类,message,name,description,发文类型,y
0,0,155000000000000,纬来体育台官方粉丝团,电视频道,中华队长林智胜 三分砲！！！\n人如其名 真的「致胜」阿～,NaN,NaN,video,H
1,3,190000000000,Mobile01,网路社群,大师兄 一发炸裂！！帅啊啊啊啊啊啊啊！！！,来自 Mobile01 贴文的相片,NaN,photo,H
2,5,242000000000000,ETtoday新闻云,新闻,爽啦～～～（ Supermm）\n\n#中华队 #古巴\nhttp://www.ettoda...,故意保送后林智胜怒轰3分炮 中华睽违9年击败古巴 | ET运动云 | ETtoday东森新闻云,世界12强棒球赛中华队林智胜，8局下在前一棒郭严文被故意保送成一、二垒有人，轰出左外野3分全...,link,H
3,6,190000000000,Mobile01,网路社群,4：1，中华战胜古巴！今晚是最棒的周末夜，谢谢中华队！请继续加油！,来自 Mobile01 贴文的相片,NaN,photo,H
4,7,109000000000000,Yahoo!奇摩新闻,新闻,一个字，爽！！！！\n\n#12强 #台古大战 #林智胜 #棒球 #三分砲 Lamigo M...,世界12强／林智胜怒轰3分砲！中华队赢古巴啦,今天(14日)「大师兄」林智胜单场被看扁2次...,link,H
...,...,...,...,...,...,...,...,...,...
6483,7824,283000000000,中天快点TV,电视频道,被镶上绿色宝石的番薯，好美！（土星人）\n\n\n---\n●CH52中天新闻On Air！...,从太空夜拍台湾美翻了！ 嘉南平原就像被镶上绿宝石,国际太空站拍下夜晚台湾，影片一上传，立刻受到热烈讨论，不少人都被台湾的美给吸引，?万家灯火构...,link,L
6484,7825,773000000000000,料理高校生,电视剧,【早鸟福利】\n方老师唯美的瞬间，就这样被我们捕捉到了 :P\n今天是11/11光棍节\n想...,动态时报相片,NaN,photo,L
6485,7826,773000000000000,料理高校生,电视剧,佩颖主任啊..\n您就别自暴自弃了~~\n \n莫非是摆脱烂桃花的新招?\n\n每周五晚间十...,动态时报相片,NaN,photo,L
6486,7827,242000000000000,ETtoday新闻云,新闻,快、合、作！（ Joe小姐）\n\n蔡依林 Jolin Tsai T.O.P. G-DRAGON,蔡依林、GD&T.O.P后台照曝光 甜呼：小爱心主人！ | 娱乐星光云 | ETtoday东...,「呸姊」蔡依林2日晚间出席南韩年度盛事MAMA舞台，她一开低胸紧身服装霸气登场，献唱舞曲《P...,link,L


In [6]:
counts50 = grp[(grp['H'] + grp['L'])>=50]
counts50['hl_score'] = counts50['H'] / (counts50['H'] + counts50['L'])
data_tmp = pd.merge(data_remove_bw, counts50, how='left', on='主页ID')
data_hl_score = data_tmp['hl_score'].fillna(0)
data_hl_score

<ipython-input-6-61051c25a3b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counts50['hl_score'] = counts50['H'] / (counts50['H'] + counts50['L'])


0       0.000000
1       0.000000
2       0.195730
3       0.000000
4       0.137514
          ...   
6483    0.063291
6484    0.064516
6485    0.064516
6486    0.195730
6487    0.039474
Name: hl_score, Length: 6488, dtype: float64

data_remove_bw['分类'] 提取 data_fl

In [7]:
data_remove_bw['分类'].value_counts()

新闻        4701
书籍杂志       754
媒体其他       244
节目         220
网路社群       219
电视频道       175
电视剧        109
电影公司        46
电影           7
广播           5
电影相关活动       4
音乐           4
Name: 分类, dtype: int64

In [8]:
# pay attentions to dataframe index
#TODO: add other categories methods.
column_mapping_str(data_remove_bw['分类'].unique())
fl_mapping = {'电视频道': 0, '网路社群': 1, '新闻': 2, '书籍杂志': 3, '电视剧': 4, '电影公司': 5, " \
             "'节目': 6, '媒体其他': 7, '电影': 5, '音乐': 7, '电影相关活动': 5, '广播': 7}
data_fl, fl_ohe = one_hot_encoder_column(data_remove_bw, '分类', fl_mapping, fill_na=7)
data_fl = pd.DataFrame(data_fl)
data_fl

,分类_0,分类_1,分类_2,分类_3,分类_4,分类_5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
6483,0.0,0.0,0.0,0.0,0.0,0.0
6484,0.0,0.0,0.0,1.0,0.0,0.0
6485,0.0,0.0,0.0,1.0,0.0,0.0
6486,0.0,1.0,0.0,0.0,0.0,0.0


## 发主类型处理
提取 data_doc_type <br>
不常见的类型归到video类 <br>
grp2 = data_remove_bw.pivot_table(index='发文类型', columns='y', values=['主页ID'], aggfunc='count').droplevel(0, axis=1)

In [9]:
doc_type_map = {'link': 1, 'photo': 2, 'video': 3}
data_doc_type, ohe_doc_type = one_hot_encoder_column(data_remove_bw, '发文类型', doc_type_map, fill_na=3)
data_doc_type


,发文类型_0,发文类型_1
0,0.0,1.0
1,1.0,0.0
2,0.0,0.0
3,1.0,0.0
4,0.0,0.0
...,...,...
6483,0.0,0.0
6484,1.0,0.0
6485,1.0,0.0
6486,0.0,0.0


## 文本类型处理
data_remove_bw[~data_remove_bw['description'].isnull()]

In [10]:
from Python.nlp.nlp import *
stop_words = get_stop_words('././data/stopwords.txt', encoding='gbk')

print(split_word("「呸姊」蔡依林2日晚间出席南韩年度盛事MAMA舞台，她一开低胸紧身服装霸气登场，献唱舞曲《P...",
                 stop_words, ['蔡依林', '叶明兰','叶镇中', '宋家豪', '白先勇','林智胜']))

total counts:  2625


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/zz/8hl03_bd6sj_wrd8__z72r8r0000gn/T/jieba.cache
Loading model cost 0.639 seconds.
Prefix dict has been built successfully.


姊 蔡依林 晚间 出席 南韩 年度 盛事 舞台 一开 低胸 紧身 服装 霸气 登场 献唱 舞曲


In [11]:
# word-vector train using all data.
data_remove_bw['sp_msg'] = data_remove_bw['description'].apply(lambda doc: split_word(doc, stop_words))
data_remove_bw[data_remove_bw['sp_msg'].map(lambda x: len(x) >= 2)]['sp_msg'].to_csv('lines.csv', index=False, header=False)

In [12]:
message_tfidf = tfidf(data_remove_bw['sp_msg'], min_df=10, output_dimension=50)
message_tfidf.head(5)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.009170,-0.023973,-0.000909,-0.035170,-0.028162,0.006321,0.003310,0.002960,0.016037,0.001375,...,-0.000055,-0.000959,-0.002332,-0.001626,-0.007754,0.001487,-0.001602,-0.003939,0.001710,-0.007521
1,-0.009170,-0.023973,-0.000909,-0.035170,-0.028162,0.006321,0.003310,0.002960,0.016037,0.001375,...,-0.000055,-0.000959,-0.002332,-0.001626,-0.007754,0.001487,-0.001602,-0.003939,0.001710,-0.007521
2,-0.046231,-0.071509,0.442372,-0.033002,0.136988,0.014411,0.002954,0.006140,-0.008024,0.009926,...,-0.039119,0.147282,-0.103097,-0.008907,0.106473,-0.040444,-0.036203,0.069273,0.050779,0.116354
3,-0.009170,-0.023973,-0.000909,-0.035170,-0.028162,0.006321,0.003310,0.002960,0.016037,0.001375,...,-0.000055,-0.000959,-0.002332,-0.001626,-0.007754,0.001487,-0.001602,-0.003939,0.001710,-0.007521
4,-0.020843,-0.036222,0.121883,-0.024897,-0.002126,0.001656,-0.005135,-0.000022,0.009108,0.009433,...,-0.005874,0.039034,-0.058400,0.025202,0.053403,-0.019719,-0.044509,0.043764,0.018796,0.087792


In [13]:
from gensim.models import word2vec
sentences = word2vec.LineSentence("./lines.csv")
model = word2vec.Word2Vec(sentences, hs=1, vector_size=50, min_count=10, window=3, sg=1)

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [19]:
data_remove_bw['msg_word2vec'] = data_remove_bw['message'].apply(lambda doc: sentence_to_vector(doc, model, stop_words, 50))
data_word2vec = pd.DataFrame(data_remove_bw['msg_word2vec'].tolist())

In [20]:
data_word2vec

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.071668,0.228472,0.157976,0.061080,-0.186352,0.047450,-0.133375,0.448490,-0.011851,-0.027702,...,0.204917,-0.134914,-0.219740,0.230358,0.340678,-0.301638,0.061941,-0.079246,-0.832016,-0.159787
1,-0.056629,0.513789,0.089883,0.024001,-0.272023,0.107954,0.002989,0.148066,-0.221473,0.243096,...,0.451101,-0.465443,0.254588,0.160884,0.250008,-0.294570,0.065925,0.017824,-0.559076,-0.050667
2,0.082722,0.207518,0.120915,-0.039144,-0.195629,0.095408,0.063435,0.386804,-0.057107,-0.069514,...,0.105105,0.019874,-0.157315,0.472120,0.350766,-0.214802,-0.186755,-0.162878,-0.831074,-0.360447
3,0.077697,0.167264,0.112328,0.003118,-0.163731,-0.081881,0.031103,0.409470,-0.113493,-0.096484,...,0.151397,-0.066600,-0.079804,0.246718,0.202615,-0.279595,-0.097795,-0.090016,-0.614861,-0.177744
4,0.222596,0.196115,0.154147,0.087344,-0.246462,-0.016380,-0.367481,0.386040,0.056479,-0.003057,...,0.227159,0.015613,-0.207617,0.359254,0.194902,-0.429804,-0.013033,-0.117133,-0.935395,-0.066576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6483,0.044299,0.308791,-0.026185,0.038236,-0.032400,-0.175670,0.102998,0.274097,-0.119307,-0.293932,...,0.084365,-0.002997,-0.024684,0.093314,0.098112,-0.271266,-0.161570,-0.087969,-0.477385,-0.126579
6484,0.007231,0.304094,0.054817,0.056769,-0.033404,-0.137759,0.120471,0.406945,-0.244273,-0.168519,...,0.213361,-0.235165,0.000221,0.038179,0.164985,-0.246952,-0.063408,-0.104404,-0.452949,-0.030019
6485,-0.006118,0.321417,-0.034239,-0.045840,-0.108783,-0.286720,-0.006618,0.125541,-0.034825,-0.440006,...,0.025443,0.049509,-0.005664,0.079750,0.262449,-0.237839,-0.240842,-0.042502,-0.538289,-0.060674
6486,0.082996,0.169294,-0.046110,0.046204,-0.001761,-0.306015,0.066896,0.231351,-0.291465,-0.335552,...,0.215147,-0.025871,0.153665,0.099969,0.089109,-0.324204,-0.243934,-0.344590,-0.288329,-0.225201


In [15]:
train_corpus = list(read_corpus("lines.csv"))

In [16]:
from gensim.models import  doc2vec
model2 = doc2vec.Doc2Vec(vector_size=50, min_count=2,epochs=40)
model2.build_vocab(train_corpus)
model2.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [21]:
message_vec = data_remove_bw['message'].apply(lambda doc: model2.infer_vector([split_word(doc, stop_words)]))
data_message = pd.DataFrame(message_vec.tolist())

In [24]:
y_mapping = {'L': 0, 'H': 1}
data_y, y_ohe = one_hot_encoder_column(data, 'y', y_mapping, fill_na=0)


In [26]:
train_data_all = pd.concat([data_hl_score,data_fl, data_doc_type, data_word2vec, data_message, data_y], axis=1)
train_data_all.to_csv("./data/train_all_1.csv")